In [9]:
import pandas as pd
import plotly.express as px
from datetime import datetime
import plotly.io as pio
pio.renderers.default = "browser"

# 🧠 Choix de granularité (modifiable dynamiquement)
granularity = "monthly"  # daily / weekly / monthly / quarterly / yearly

# 🧠 Fonction pour configurer la vue x-axis selon la granularité
def get_time_settings(granularity):
    if granularity == "daily":
        return {"tickformat": "%d %b", "dtick": "D1"}
    elif granularity == "weekly":
        return {"tickformat": "Semaine %W\n%Y", "dtick": "M1"}
    elif granularity == "monthly":
        return {"tickformat": "%b %Y", "dtick": "M1"}
    elif granularity == "quarterly":
        return {"tickformat": "T%q %Y", "dtick": "M3"}
    elif granularity == "yearly":
        return {"tickformat": "%Y", "dtick": "M12"}
    else:
        return {}

# 🗂️ Charger les données
df = pd.read_excel("all_clickup_tasks_with_subtasks.xlsx")
df = df.dropna(subset=["start_date", "due_date"])

rows = []

# 🔁 Créer hiérarchie visuelle : Projet → Task → Subtask
for list_name in df["list"].unique():
    group = df[df["list"] == list_name]
    start_proj = pd.to_datetime(group["start_date"]).min()
    end_proj = pd.to_datetime(group["due_date"]).max()
    rows.append({
        "y_label": f"📦 {list_name}",
        "start": start_proj,
        "end": end_proj,
        "level": 0,
        "project": list_name
    })

    tasks = group[group["type"] == "task"].sort_values("start_date")
    for _, task_row in tasks.iterrows():
        task_id = task_row["task_id"]
        rows.append({
            "y_label": f"   📌 {task_row['task_name']} [{task_row['status']}]",
            "start": pd.to_datetime(task_row["start_date"]),
            "end": pd.to_datetime(task_row["due_date"]),
            "level": 1,
            "project": list_name
        })

        subtasks = group[(group["type"] == "subtask") & (group["parent_id"] == task_id)].sort_values("start_date")
        for _, sub_row in subtasks.iterrows():
            rows.append({
                "y_label": f"      └🧷 {sub_row['task_name']} [{sub_row['status']}]",
                "start": pd.to_datetime(sub_row["start_date"]),
                "end": pd.to_datetime(sub_row["due_date"]),
                "level": 2,
                "project": list_name
            })

# 🧱 Conversion
timeline_df = pd.DataFrame(rows)

# 🧭 Créer le graphique
fig = px.timeline(
    timeline_df,
    x_start="start",
    x_end="end",
    y="y_label",
    color="project",
    title=f"🗓️ Vue Timeline ({granularity.capitalize()})"
)

# 🧷 Inversion axe Y
fig.update_yaxes(autorange="reversed")

# 🎯 Config vue temporelle
time_config = get_time_settings(granularity)
fig.update_xaxes(**time_config)

# 🎨 Mise en forme générale
fig.update_layout(
    height=900,  # ✅ Réduction de hauteur ici
    margin=dict(l=300, r=50, t=80, b=50),
    legend_title="Projet"
)

fig.show()
